<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="-unSkript-Runbooks-">unSkript Runbooks&nbsp;</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective"><strong>Objective</strong></h3>
<strong>Get publicly accessible AWS RDS instances using unSkript actions.</strong></div>
</center><center>
<h2 id="Publicly-Accessible-Amazon-RDS-Instances">Publicly Accessible Amazon RDS Instances</h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<ol>
<li>AWS Get Publicly Accessible RDS Instances</li>
<li>Post Slack Message</li>
</ol>

In [4]:
if region == None:
    region = ""

<h3 id="AWS-Get-Publicly-Accessible-RDS-Instances"><a id="1" target="_self" rel="nofollow"></a>AWS Get Publicly Accessible RDS Instances</h3>
<p>Using unSkript's <strong>AWS Get Publicly Accessible RDS Instances</strong>&nbsp;action, we will get all the publicly accessible instances from RDS instances.</p>
<blockquote>
<p>Input parameters: <code>region</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>rds_instances</code></p>
</blockquote>

In [5]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
import pprint
from typing import Optional, Tuple
from pydantic import BaseModel, Field
from unskript.legos.utils import CheckOutput
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
from unskript.connectors.aws import aws_get_paginator


from beartype import beartype
@beartype
def aws_get_publicly_accessible_db_instances_printer(output):
    if output is None:
        return

    if isinstance(output, CheckOutput):
        print(output.json())
    else:
        pprint.pprint(output)


@beartype
def aws_get_publicly_accessible_db_instances(handle, region: str = "") -> Tuple:
    """aws_get_publicly_accessible_db_instances Gets all publicly accessible DB instances

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: Region of the RDS.

        :rtype: CheckOutput with status result and list of publicly accessible RDS instances.
    """
    result = []
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)
    for reg in all_regions:
        try:
            ec2Client = handle.client('rds', region_name=reg)
            response = aws_get_paginator(ec2Client, "describe_db_instances", "DBInstances")
            for db in response:
                db_instance_dict = {}
                if db['PubliclyAccessible']:
                    db_instance_dict["region"] = reg
                    db_instance_dict["instance"] = db['DBInstanceIdentifier']
                    result.append(db_instance_dict)
        except Exception:
            pass

    if len(result) != 0:
        return (False, result)
    return (True, None)


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region"
    }''')

task.configure(outputName="rds_instances")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_publicly_accessible_db_instances, lego_printer=aws_get_publicly_accessible_db_instances_printer, hdl=hdl, args=args)

<h3 id="Modify-Output">Modify Output</h3>
<p>In this action, we modify the output from step 1 and return a list of dictionary items for the publicly accessible RDS instances.</p>
<blockquote>
<p><strong>Output variable:</strong> rds_instance_list</p>
</blockquote>

In [5]:
rds_instance_list = []
try:
    if rds_instances:
        if rds_instances[0] == False:
            for instance in rds_instances[1]:
                rds_instance_list.append(instance)

except Exception as e:
    pass

<h3 id="Post-Slack-Message">Post Slack Message</h3>
<p>Here we will use unSkript <strong>Post Slack Message</strong> action. This action takes channel: str and message: str as input. This input is used to post the message to the slack channel.</p>
<blockquote>
<p><strong>Input parameters:</strong> <code>message</code>, <code>channel</code></p>
</blockquote>
<blockquote>
<p><strong>Output variable</strong>: <code>message_status</code></p>
</blockquote>

In [14]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)

from beartype import beartype

from beartype import beartype
@beartype
def slack_post_message_printer(output):
    if output is not None:
        pprint.pprint(output)
    else:
        return


@beartype
@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> str:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return f"Successfuly Sent Message on Channel: #{channel}"
    except SlackApiError as e:
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        if e.response['error'] == 'channel_not_found':
            raise Exception('Channel Not Found')
        elif e.response['error'] == 'duplicate_channel_not_found':
            raise Exception('Channel associated with the message_id not valid')
        elif e.response['error'] == 'not_in_channel':
            raise Exception('Cannot post message to channel user is not in')
        elif e.response['error'] == 'is_archived':
            raise Exception('Channel has been archived')
        elif e.response['error'] == 'msg_too_long':
            raise Exception('Message text is too long')
        elif e.response['error'] == 'no_text':
            raise Exception('Message text was not provided')
        elif e.response['error'] == 'restricted_action':
            raise Exception('Workspace preference prevents user from posting')
        elif e.response['error'] == 'restricted_action_read_only_channel':
            raise Exception('Cannot Post message, read-only channel')
        elif e.response['error'] == 'team_access_not_granted':
            raise Exception('The token used is not granted access to the workspace')
        elif e.response['error'] == 'not_authed':
            raise Exception('No Authtnecition token provided')
        elif e.response['error'] == 'invalid_auth':
            raise Exception('Some aspect of Authentication cannot be validated. Request denied')
        elif e.response['error'] == 'access_denied':
            raise Exception('Access to a resource specified in the request denied')
        elif e.response['error'] == 'account_inactive':
            raise Exception('Authentication token is for a deleted user')
        elif e.response['error'] == 'token_revoked':
            raise Exception('Authentication token for a deleted user has been revoked')
        elif e.response['error'] == 'no_permission':
            raise Exception('The workspace toekn used does not have necessary permission to send message')
        elif e.response['error'] == 'ratelimited':
            raise Exception('The request has been ratelimited. Retry sending message later')
        elif e.response['error'] == 'service_unavailable':
            raise Exception('The service is temporarily unavailable')
        elif e.response['error'] == 'fatal_error':
            raise Exception('The server encountered catostrophic error while sending message')
        elif e.response['error'] == 'internal_error':
            raise Exception('The server could not complete operation, likely due to transietn issue')
        elif e.response['error'] == 'request_timeout':
            raise Exception('Sending message error via POST: either message was missing or truncated')
        else:
            raise Exception(f'Failed Sending Message to slack channel {channel} Error: {e.response["error"]}')

    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return f"Unable to send message on {channel}"


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "channel": "channel",
    "message": "f\\"Publicly Accessible RDS Instances: {rds_instance_list}\\""
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "channel",
    "condition_result": true
    }''')
task.configure(outputName="message_status")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(slack_post_message, lego_printer=slack_post_message_printer, hdl=hdl, args=args)

### Conclusion
<p>In this Runbook, we demonstrated the use of unSkript's AWS actions. This runbook help to find Publicly Accessible RDS Instances and post a slack message to the given channel. To view the full platform capabilities of unSkript please visit&nbsp;<a href="https://us.app.unskript.io" target="_blank" rel="noopener">https://us.app.unskript.io</a></p>